In [2]:
from itertools import product, combinations
import numpy as np
import networkx as nx
from icecream import ic

In [3]:
def create_problem(
    size: int,
    *,
    density: float = 1.0,
    negative_values: bool = False,
    noise_level: float = 0.0,
    seed: int = 42,
) -> np.ndarray:
    """Problem generator for Lab3"""
    rng = np.random.default_rng(seed)
    map = rng.random(size=(size, 2))
    problem = rng.random((size, size))
    if negative_values:
        problem = problem * 2 - 1
    problem *= noise_level
    for a, b in product(range(size), repeat=2):
        if rng.random() < density:
            problem[a, b] += np.sqrt(
                np.square(map[a, 0] - map[b, 0]) + np.square(map[a, 1] - map[b, 1])
            )
        else:
            problem[a, b] = np.inf
    np.fill_diagonal(problem, 0)
    return (problem * 1_000).round()

In [4]:
problem = create_problem(10, density=0.8, noise_level=10, negative_values=False)

In [5]:
masked = np.ma.masked_array(problem, mask=np.isinf(problem))
G = nx.from_numpy_array(masked, create_using=nx.DiGraph)

In [ ]:
for s, density in combinations(range(problem.shape[0]), 2):
    try:
        # path = nx.shortest_path(G, s, d, weight='weight')
        path = nx.bellman_ford_path(G, s, density, weight='weight')
        cost = nx.path_weight(G, path, weight='weight')
    except nx.NetworkXNoPath:
        # Nodes are not connected
        path = None
        cost = np.inf
    except nx.NetworkXUnbounded:
        # Negative cycle detected
        path = None
        cost = -np.inf
    ic(s, density, path, cost)
None

In [7]:
import heapq
def dijkstra_shortest_path(G, source, target):
    pq = [(0, source)]
    distances = {node: float('inf') for node in G.nodes}
    distances[source] = 0
    previous_nodes = {node: None for node in G.nodes}
    while pq:
        current_distance, current_node = heapq.heappop(pq)
        if current_distance > distances[current_node]:
            continue
        for neighbor in G.neighbors(current_node):
            weight = G[current_node][neighbor]['weight']
            distance = current_distance + weight
            if distance < distances[neighbor]:
                distances[neighbor] = distance
                previous_nodes[neighbor] = current_node
                heapq.heappush(pq, (distance, neighbor))
    path = []
    current_node = target
    while current_node is not None:
        path.append(current_node)
        current_node = previous_nodes[current_node]
    path.reverse()
    return path, distances[target]

def bellman_ford_shortest_path(G, source, target):
    distances = {node: float('inf') for node in G.nodes}
    distances[source] = 0
    previous_nodes = {node: None for node in G.nodes}
    for _ in range(len(G.nodes) - 1):
        for u, v, data in G.edges(data=True):
            weight = data['weight']
            if distances[u] + weight < distances[v]:
                distances[v] = distances[u] + weight
                previous_nodes[v] = u
    for u, v, data in G.edges(data=True):
        weight = data['weight']
        if distances[u] + weight < distances[v]:
            return None, -np.inf
    path = []
    current_node = target
    while current_node is not None:
        path.append(current_node)
        current_node = previous_nodes[current_node]
    path.reverse()
    return path, distances[target]

In [ ]:
s = [10, 20, 50, 100, 200, 500, 1000]
density = [0.2, 0.5, 0.8, 1.0]
noise_level = [0.0, 0.1, 0.5, 0.8]
negative_values = [False, True]

In [ ]:
import os
def process_single_problem(params):
    s, d, n, neg = params

    if os.path.exists(f'problems/problem_size-{s}_density-{d}_noise-{n}_negative-{neg}.csv'):
        print(f'Skipping existing problem size-{s}_density-{d}_noise-{n}_negative-{neg}')
        return f'problems/problem_size-{s}_density-{d}_noise-{n}_negative-{neg}.csv'
    
    problem = create_problem(
        s, density=d, noise_level=n, negative_values=neg, seed=42
    )

    masked = np.ma.masked_array(problem, mask=np.isinf(problem))
    G = nx.from_numpy_array(masked, create_using=nx.DiGraph)
    

    rng = np.random.default_rng(42)
    results = []
    
    # Run with 5 random start-end pairs
    for _ in range(5):
        start = rng.integers(0, s)
        end = rng.integers(0, s)
        while start == end:  # Ensure start != end
            end = rng.integers(0, s)
            
        if neg:
            path, cost = bellman_ford_shortest_path(G, start, end)

            try:
                nx_path = nx.bellman_ford_path(G, start, end, weight='weight')
                nx_cost = nx.path_weight(G, nx_path, weight='weight')
            except nx.NetworkXNoPath:
                nx_path = None
                nx_cost = np.inf
            except nx.NetworkXUnbounded:
                nx_path = None
                nx_cost = -np.inf

        else:
            try:
                path, cost = dijkstra_shortest_path(G, start, end)
                nx_path = nx.dijkstra_path(G, start, end, weight='weight')
                nx_cost = nx.path_weight(G, nx_path, weight='weight')
            except nx.NetworkXNoPath:
                path = None
                cost = np.inf
                nx_path = None
                nx_cost = np.inf
        
        results.append([start, end, path, cost, nx_path, nx_cost])
    
    
    # Write results to CSV with headers
    filename = f'problems/problem_size-{s}_density-{d}_noise-{n}_negative-{neg}.csv'
    with open(filename, 'w') as f:
        # Write header
        f.write('start,end,my_path,my_cost,nx_path,nx_cost\n')
        
        # Write data
        for result in results:
            f.write(','.join([str(x) for x in result]) + '\n')
    
    return filename

# Cartesian product of all parameters
param_combinations = list(product(s, density, noise_level, negative_values))

for params in param_combinations:
    filename = process_single_problem(params)


In [ ]:
def create_problem_for_astar(
    size: int,
    density: float = 1.0,
    negative_values: bool = False,
    noise_level: float = 0.0,
    seed: int = 42,
) -> np.ndarray:
    """Problem generator for Lab3"""
    rng = np.random.default_rng(seed)
    map = rng.random(size=(size, 2))
    problem = rng.random((size, size))
    if negative_values:
        problem = problem * 2 - 1
    problem *= noise_level
    for a, b in product(range(size), repeat=2):
        if rng.random() < density:
            problem[a, b] += np.sqrt(
                np.square(map[a, 0] - map[b, 0]) + np.square(map[a, 1] - map[b, 1])
            )
        else:
            problem[a, b] = np.inf
    np.fill_diagonal(problem, 0)
    return (problem * 1_000).round(), map

In [ ]:
def astar_shortest_path(G, source, target, node_positions):
    """
    A* with euclidean distance as heuristic
    """
    def heuristic(node1, node2):
        """Euclidean distance between two nodes"""
        x1, y1 = node_positions[node1]
        x2, y2 = node_positions[node2]
        return np.sqrt((x1 - x2)**2 + (y1 - y2)**2) * 1000  # Scaled to match problem weights
    
    # PQ by f_score (g_score + heuristic)
    pq = [(heuristic(source, target), source)]
    
    # Cost from start to each node (g_score)
    g_score = {node: float('inf') for node in G.nodes}
    g_score[source] = 0
    
    previous_nodes = {node: None for node in G.nodes}

    visited = set()
    
    while pq:
        current_f, current_node = heapq.heappop(pq)
        
        if current_node in visited:
            continue
            
        visited.add(current_node)
        
        if current_node == target:
            path = []
            while current_node is not None:
                path.append(current_node)
                current_node = previous_nodes[current_node]
            path.reverse()
            return path, g_score[target]
        
        for neighbor in G.neighbors(current_node):
            if neighbor in visited:
                continue
                
            edge_weight = G[current_node][neighbor]['weight']
            tentative_g_score = g_score[current_node] + edge_weight
            
            if tentative_g_score < g_score[neighbor]:
                # This path to neighbor is better than any previous one
                previous_nodes[neighbor] = current_node
                g_score[neighbor] = tentative_g_score
                f_score = tentative_g_score + heuristic(neighbor, target)
                heapq.heappush(pq, (f_score, neighbor))
    
    # No path found
    return None, float('inf')

In [22]:
size = [10, 20, 50, 100, 200, 500, 1000]
density = [0.2, 0.5, 0.8, 1.0]
noise_level = [0.0, 0.1, 0.5, 0.8]

In [ ]:
from itertools import product
import random
from tqdm import tqdm

h_fun = lambda u, v: np.sqrt(
            (node_positions[u][0] - node_positions[v][0])**2 +
            (node_positions[u][1] - node_positions[v][1])**2) * 1000

results = []


for s, density, noise in tqdm(list(product(size, density, noise_level)), desc='Processing A* problems'):
    problem, node_positions = create_problem_for_astar(
        s, density=density, noise_level=noise, negative_values=False, seed=42
    )
    masked = np.ma.masked_array(problem, mask=np.isinf(problem))
    G = nx.from_numpy_array(masked, create_using=nx.DiGraph)

    start, end = random.randint(0, s-1), random.randint(0, s-1)
    while start == end:
        end = random.randint(0, s-1)


    path, cost = astar_shortest_path(G, start, end, node_positions)

    try:
        nx_path = nx.astar_path(G, start, end, heuristic=h_fun, weight='weight')
        nx_cost = nx.path_weight(G, nx_path, weight='weight')
    except nx.NetworkXNoPath:
        nx_path = None
        nx_cost = np.inf
    # ic(s, e, path, cost, nx_path, nx_cost)


    results.append((s, density, noise, s, end, path, cost, nx_path, nx_cost))


header = 'size,density,noise,start,end,my_cost,nx_cost,my_path,nx_path\n'
with open('astar/astar_results.csv', 'w') as f:
    f.write(header)
    for s, density, noise, start, end, path, cost, nx_path, nx_cost in results:
        f.write(f"{s},{density},{noise},{start},{end},{cost},{nx_cost},{path},{nx_path}\n")

    

Processing A* problems: 100%|██████████| 112/112 [02:05<00:00,  1.12s/it]
